In [1]:
import os
import openai
import tiktoken


import xml.etree.ElementTree as ET
import networkx as nx
import matplotlib.pyplot as plt


import json
import networkx as nx

In [3]:
import os
import openai


from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file

openai.api_key  = os.environ['OPENAI_API_KEY']

In [13]:
# import os
# from openai import OpenAI

# client = OpenAI(
#     # This is the default and can be omitted
#     api_key=os.environ.get("OPENAI_API_KEY"),
# )

In [6]:
# print(client)

In [7]:
# Path to the roles list
path_to_id = ["protocolSection","identificationModule","nctId"]
path_to_interventions = ["protocolSection", "armsInterventionsModule", "interventions"]
path_to_description = ["protocolSection","descriptionModule","briefSummary"]

In [8]:
# Function to retrieve a value from a JSON structure based on a given path
def get_value_from_path(data, path):
    """
    Retrieve the value from a JSON structure based on a given path.

    :param data: The JSON data (could be a dictionary, list, or basic data type).
    :param path: A list of keys/indices that define the path to the desired value.
    :return: The value at the specified path, or None if the path doesn't exist.
    """
    current = data

    try:
        for key in path:
            if isinstance(current, dict):
                current = current[key]
            elif isinstance(current, list) and isinstance(key, int):
                current = current[key]
            else:
                return None  # Path leads to an unexpected type
        return current
    except (KeyError, IndexError):
        return None  # If the key/index does not exist

In [9]:
import os
import json

# Define the directory containing the JSON files
# directory_path = "../CancerDrugCombo/datasets/drug-comb-nct/"
directory_path = "../datasets/drug-comb-nct/"

count = 0

nct_drug_combos = {}

# Iterate over each file in the directory
for filename in os.listdir(directory_path):
    # Check if the file is a JSON file
    if filename.endswith(".json"):
        file_path = os.path.join(directory_path, filename)  # Get the full path to the file
        
        # Open and read the JSON file
        with open(file_path, 'r') as json_file:
            # Parse the JSON content
            data = json.load(json_file)
            
            # Get the roles list
            desc = get_value_from_path(data, path_to_description)
            nctid = get_value_from_path(data, path_to_id)
            
            # print(nctid, desc)
            
            nct_drug_combos[nctid] = desc

In [10]:
len(nct_drug_combos)

2496

In [11]:
list(nct_drug_combos.values())[0]

'In order to prevent the high mortality due to an hypothetic pandemic caused by a newly emerging influenza A virus, antiviral drugs are seen as essential requirements for control of initial influenza outbreaks.Two antivirals are available for the treatment oseltamivir and zanamivir. Emergence of Oseltamivir resistance has been recently reported. . It appeared opportune to assess the efficacy and safety of biotherapy of neuraminidase inhibitors ,will be investigated by a randomized, placebo controlled, double blind study in France, during the next winter season . This study will be conducted in 300 centres of primary care with 900 adults with a virologically suspected influenza A infection. Individuals will be randomized to 1 of the 3 treatment groups: oseltamivir +zanamivir, or oseltamivir+placebo or placebo +zanamivir.The primary judgment criteria will be the proportion of patients with negative RT PCR negative in nasal secretions at Day 2.'

In [12]:
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file

openai.api_key  = os.environ['OPENAI_API_KEY']

# Upload a file
from openai import OpenAI
client = OpenAI()

In [14]:
# xml_pattern = """
# <?xml version="1.0" encoding="UTF-8"?>
# <network>
#     <nodes>
#         <node label="tocilizumab (RoActemra)"/>
#         <node label="methotrexate"/>
#         <node label="DMARDs"/>
#     </nodes>
#     <edges>
#         <edge source="tocilizumab (RoActemra)" target="methotrexate" label="combination"/>
#         <edge source="tocilizumab (RoActemra)" target="DMARDs" label="combination"/>
#         <edge source="tocilizumab (RoActemra)" target="methotrexate" label="combination"/>
#         <edge source="tocilizumab (RoActemra)" target="DMARDs" label="combination"/>
#     </edges>
# </network>
# """

In [15]:
shot_1 = """is to compare the safety and pharmacokinetics of <CKD-330> (fixed-dose [combination] of <Candesartan> 16 mg and <Amlodipine> 5 mg)"""
shot_2 = """ <oseltamivir> [+] <zanamivir>""",
shot_3 = """ to predict the efficacy and safety of <colistin-rifampin> [combination] and colistin monotherapy in extensively drug-resistant acinetobacter baumannii""",
shot_4 = """versus <docetaxel> [in combination with] either <gemcitabine> or <vinorelbine> or in [combination] with <capecitabine>.""",
shot_5 = """<GW642444M> and <GW685698X> will be simultaneously [co-administered] from a single device and compared with <GW642444M> [and] <GW685698X> administered separately in order to determine whether [co-administration] affects the safety""",
shot_6 = """fixed [combination] product <QVA149> versus the component products <QAB149> and <NVA237>, [administered concurrently] """,
shot_7 = """The purpose of this study is to find out whether or not [the addition of] <hydroxyurea> to <didanosine (ddI)> and other anti-HIV medications"""

pe_shots = [
    shot_1,
    shot_2,
    shot_3,
    shot_4, 
    shot_5,
    shot_6,
    shot_7
]

In [27]:
comb_ex1 = "tocilizumab (RoActemra)|methotrexate"
comb_ex2 = "tocilizumab (RoActemra)|DMARDs"

In [40]:
count = 0
file_name = '../outputs/pe_nct_combs_found/drug-combos-found-in-nct.txt'
for nct_id, nct_desc in nct_drug_combos.items():

    prompt = f""" 

        You are an specialized drug annotator, detect drugs if annotated/marked using < and > and relationship if it is annotated/maked using [ and ] \

        Your task is to learn from the following few shots:
    
            '''{shot_1}''' \
            '''{shot_2}''' \
            '''{shot_3}''' \
            '''{shot_4}''' \
            '''{shot_5}''' \
            '''{shot_6}''' \
            '''{shot_7}''' \

        Now you need to analyze the decription this clinical trials '''{nct_desc}''' to identify drugs and potential combinations \
    
        Your response will combinations discovered in pipe-delimited format that follows the following examples:
                        
            '''{nct_id} | {comb_ex1}'''             
            '''{nct_id} | {comb_ex2}''' 
            write each combination found in a single line and not other messages should be written
    """

    response = client.chat.completions.create(
        messages=[
            {
                "role": "user",
                "content": prompt,
            }
        ],
        model="gpt-3.5-turbo",
    )


    # Extract the completed text correctly
    combos_discovered = response.choices[0].message.content.strip()
    if not combos_discovered.lower().startswith('no') and not combos_discovered.lower().startswith('n/a'):
        count += 1
        with open(file_name, 'a') as pipes_file:
            pipes_file.write(combos_discovered)
            pipes_file.write('\n')
        
    if count % 200 == 0:
        print("Current NCT: ", nct_id)

Current NCT:  NCT01612741
Current NCT:  NCT00006208
Current NCT:  NCT00191438
Current NCT:  NCT04080895
Current NCT:  NCT01618214
Current NCT:  NCT00016276
Current NCT:  NCT00050908
Current NCT:  NCT05066841
Current NCT:  NCT03875508
Current NCT:  NCT05259696
Current NCT:  NCT00473616


In [ ]:
# # Get a response from the chat-based model
# xml_response = chat_with_gpt(prompt)

In [ ]:
count = 2496



for marked_sent in list(nct_drug_combos.values())[2496:]:


    
    """    
    
    
    # Get a response from the chat-based model
    xml_response = chat_with_gpt(prompt)
    
    # Re-parse the XML data
    try:
        root = ET.fromstring(xml_response)  # Ensure XML data is well-formed
    except ET.ParseError as e:
        print(xml_response)
        print("XML Parsing Error:", e)
    
    # Add nodes to the graph
    nodes = root.find("Nodes")
    for node in nodes:
        node_name = node.get("name")
        xmlG.add_node(node_name)

    # Add links to the graph
    links = root.find("Links")
    for link in links:
        source = link.get("source")
        target = link.get("target")
        relationship = link.get("relationship")
        xmlG.add_edge(source, target, relationship=relationship)
    
    count +=1 
    print("Current count: ", count)
    if count % 100 == 0:
        
        print("Current network: ", xmlG)
        print("---------------------")    

In [ ]:
count = 2496


for nct_desc in list(nct_drug_combos.values())[2496:]:

prompt = f""" using the pattern {xml_pattern} \

For an output example to be produced in XML format \

What the mighty and pwerful chatgpt do if I am summoned 
it to identify only drugs from the following text  {nct_desc} just prompt-engineering?  \

Make them output as a network of nodes and edges """    


# Get a response from the chat-based model
xml_response = chat_with_gpt(prompt)
    
    # Re-parse the XML data
    try:
        root = ET.fromstring(xml_response)  # Ensure XML data is well-formed
    except ET.ParseError as e:
        print(xml_response)
        print("XML Parsing Error:", e)
    
    # Add nodes to the graph
    nodes = root.find("Nodes")
    for node in nodes:
        node_name = node.get("name")
        xmlG.add_node(node_name)

    # Add links to the graph
    links = root.find("Links")
    for link in links:
        source = link.get("source")
        target = link.get("target")
        relationship = link.get("relationship")
        xmlG.add_edge(source, target, relationship=relationship)
    
    count +=1 
    print("Current count: ", count)
    if count % 100 == 0:
        
        print("Current network: ", xmlG)
        print("---------------------")    

In [ ]:
print(xmlG)

In [ ]:
import pickle

# Save the graph to a pickle file
pickle_file_path = "../datasets/drug_combo_graph.pkl"
with open(pickle_file_path, "wb") as pickle_file:
    pickle.dump(xmlG, pickle_file)

print(f"Graph has been saved to {pickle_file_path}")

In [ ]:
import pickle

In [ ]:
# Load the graph from the pickle file
with open("../datasets/drug_combo_graph.pkl", "rb") as pickle_file:
    loaded_graph = pickle.load(pickle_file)

In [ ]:
print(loaded_graph)

In [ ]:
# Graph with 3629 nodes and 4263 edges, combination therapy Relationships:  2873
# Other Relations:  1390

In [ ]:
# import networkx as nx

# # Calculate degree centrality
# degree_centrality = nx.degree_centrality(loaded_graph)
# # Sort nodes by degree centrality in descending order
# sorted_degree_centrality = sorted(degree_centrality.items(), key=lambda x: x[1], reverse=True)

# # Get the top 20 drugs by degree centrality
# top_50_drugs = [node for node, centrality in sorted_degree_centrality[:50]]

# print("Top 20 Drugs by Degree Centrality:", top_50_drugs)


In [ ]:
# copied_loaded_graph = loaded_graph

In [ ]:
# ### parse CHEBI DRUGS
# def load_chebi_drugs(chebi_file):
    
#     drugs_dict = {}
    
#     handler = open(chebi_file)
#     lines = handler.readlines()
    
#     drid = ''
#     dname = ''
#     for line in lines[17:]:
#         if not line.startswith('[Term]'): 
#             if line.startswith('id: '):
#                 drid = line.rstrip()[3:].strip()
                
#             elif line.startswith('name: '):
#                 dname = line.rstrip().strip('\n')[6:]  
#                 drugs_dict[drid] = dname.lower()          
#     return drugs_dict 

In [ ]:
# chebi_file = '../onto/chebi.obo'

# chem_entities = load_chebi_drugs(chebi_file)

In [ ]:
# # load the drugs from the graph of combinations
# chebi_drugs = list(chem_entities.values())

In [ ]:
# chebi_drugs[:10]

In [ ]:
# Now remove all nodes identified for removal
# for node in nodes_to_remove:
#     copied_loaded_graph.remove_node(node)

# print("Nodes after removal:", list(copied_loaded_graph.nodes()))

In [ ]:
print(loaded_graph)

In [ ]:
combo_rel = 0
else_rel = 0
for edge in loaded_graph.edges(data=True):
    if edge[2]['relationship'] == 'combination therapy':
        combo_rel += 1
    else:
        else_rel += 1
print('combo_rel: ', combo_rel)
print('else_rel: ', else_rel)

In [ ]:
import openai

# Define the prompt with the masked sentences
prompt = """
The cat chased the mouse.
The student studied for the exam.
The chef cooked the delicious meal.
The gardener watered the plants.
The firefighter extinguished the fire.
The parent hugged the child.
The engineer built the bridge.

If a cat "chased" a mouse, and engineer "built" a bridge,

and

the parent "hugged a child"

Unmask the XXX with proper verb from the following sentences:

authors XXX novel
scientist XXX cure
"""

# Generate completions using GPT-3.5 chat model
response = openai.ChatCompletion.create(
    model="text-davinci-003",  # Choose an available model
    messages=[
        {"role": "system", "content": "You are a helpful assistant who can discover relationships."},
        {"role": "user", "content": prompt}
    ]
)

# Extract the completed text
completed_text = response.choices[0].message["content"].strip()

print(completed_text)
